#### Relevant Imports

In [1]:
from sqlalchemy import create_engine, text
import json
from collections import defaultdict
from typing import List, Dict, Any
import csv
from groq import Groq
import dotenv


**SQLite DB**  
Let us connect to the SQLite Sample Database what we have  


In [2]:
# There is an engine instance created, which can handle multiple connetions
sql_engine = create_engine("sqlite:///Sample_2 - Copy.db")
conn_1 = sql_engine.connect ()

**Get Query**  
Given a user prompt and table schema, LLM can generate SQL query  
The query generation requires specific instructions, so that it can be used correctly

In [3]:
dotenv.load_dotenv ()
client = Groq()

In [ ]:
Q_Instr = "From the given SQL table schema, formulate SQL query which answers user's question.\
        Respond only the SQL query. No title, no introduction."

Schema = """
            CREATE TABLE Student_Performance (
            student_id VARCHAR(50),
            age INTEGER,
            gender VARCHAR(50),
            study_hours_per_day REAL,
            social_media_hours REAL,
            netflix_hours REAL,
            part_time_job VARCHAR(50),
            attendance_percentage REAL,
            sleep_hours REAL,
            diet_quality VARCHAR(50),
            exercise_frequency INTEGER,
            parental_education_level VARCHAR(50),
            internet_quality VARCHAR(50),
            mental_health_rating INTEGER,
            extracurricular_participation VARCHAR(50),
            exam_score REAL
          )
        """

Prompt = "On an average students watch netflix for ..."
# Prompt = "Are students taking enough rest?"
# Prompt = "Is there a correlation between study time and score really?"
# Prompt = "Tell me top 5 scorer who are doing a job additionally, because its quite challenging"

messages=[
    {
        "role": "system",
        "content": Q_Instr
    },

    {
        "role": "user",
        "content": "Schema :\n"+Schema+"\n Question : \n"+Prompt
    }
]
completion = client.chat.completions.create(
    messages=messages,
    model="llama3-70b-8192",

)

print (completion.choices[0].message.content)

>Additional instructions to fine tune the query generation  
>This will help to get precise output.

In [ ]:
Q_Instr = "From the given SQL table schema, formulate SQL query which answers user's question.\
        Respond only the SQL query. No title, no introduction. Give complete query.\
        Its SQLite DB.\
        When searching text field, always bring to lower case and use wild card : %xxx% .\
        If question is not relevant to schema, say 'No relevant data'"


Schema = """
            CREATE TABLE Student_Performance (
            student_id VARCHAR(50),
            age INTEGER,
            gender VARCHAR(50),
            study_hours_per_day REAL,
            social_media_hours REAL,
            netflix_hours REAL,
            part_time_job VARCHAR(50),
            attendance_percentage REAL,
            sleep_hours REAL,
            diet_quality VARCHAR(50),
            exercise_frequency INTEGER,
            parental_education_level VARCHAR(50),
            internet_quality VARCHAR(50),
            mental_health_rating INTEGER,
            extracurricular_participation VARCHAR(50),
            exam_score REAL
          )
        """

Prompt = "Is there a correlation between study time and score really?"
# Prompt = "Tell me top 5 scorers who are doing a job additionally, because its quite challenging"
# Prompt = "What are the kind of diet habits they have?"
# Prompt = "When are the exams?"

messages=[
    {
        "role": "system",
        "content": Q_Instr
    },

    {
        "role": "user",
        "content": "Schema :\n"+Schema+"\n Question : \n"+Prompt
    }
]
completion = client.chat.completions.create(
    messages=messages,
    model="llama3-70b-8192",   

)

Query_String = completion.choices[0].message.content

print (Query_String)

**Complete Pipeline**
The query string further needs to be used for extraction, provide the context from there and get the answer from LLM  
First invoke of LLM to get the SQL Query. Then next one for getting the response

In [ ]:
Prompt = "how many students are studying more?"
# Prompt = "Who are the top 5 scorers and how is their social media, netflix habits?"
# Prompt = "Considering the top 10 students, do you think social media time and netflix matters?"
# Prompt = "students whose parents have completed High school, have scored more than 90 in exams"


messages=[
    {
        "role": "system",
        "content": Q_Instr
    },

    {
        "role": "user",
        "content": "Schema :\n"+Schema+"\n Question : \n"+Prompt
    }
]
completion = client.chat.completions.create(
    messages=messages,
    model="llama3-70b-8192",   

)

## Get the query string
Query_String = completion.choices[0].message.content
print(Query_String)

#Fetch the data from DB
result = conn_1.execute (text(Query_String))

# Query output into JSON
rows = [row._asdict () for row in result]
json_output = json.dumps(rows,  indent=2)

R_Instr = "Using the context given, provide response to the user question or statement.\
            Context is provided as JSON information.\
            Answer to the question precisely"

messages=[
    {
        "role": "system",
        "content": R_Instr
    },

    {
        "role": "user",
        "content":"Context : \n"+ json_output + "Query : \n" + Prompt
    }
]
completion = client.chat.completions.create(
    messages=messages,
    model="llama3-70b-8192",
)

print (completion.choices[0].message.content)